In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [47]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [48]:
train_data_path = "./train/"

transforms = transforms.Compose(
    [
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], # mean and sd of ImageNet dataset
                              std=[0.229, 0.224, 0.225])
    ]
)
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms, is_valid_file=check_image)

In [49]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms, is_valid_file=check_image)

test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms, is_valid_file=check_image)

In [50]:
from torch.utils import data

batch_size = 64
train_data_loader = data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

In [51]:
from torch import nn
import torch.nn.functional as F


class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84) # 12288 = 64*64*3. Second number is fairly arbritrary
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2) # last number has to be 2 since there are 2 classes

    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

simplenet = SimpleNet()

In [52]:
device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [53]:
import torch.optim as optim

optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [54]:
def train(model, optimizer: optim.Adam, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [55]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 3.64, Validation Loss: 8.06, accuracy = 0.31
Epoch: 2, Training Loss: 4.51, Validation Loss: 1.71, accuracy = 0.67
Epoch: 3, Training Loss: 1.22, Validation Loss: 1.44, accuracy = 0.46
Epoch: 4, Training Loss: 1.00, Validation Loss: 0.80, accuracy = 0.74
Epoch: 5, Training Loss: 0.51, Validation Loss: 0.71, accuracy = 0.67


## Making predictions

In [56]:
labels = ["cat", "fish"]

img = Image.open("./test/cat/6170850_a262e64099.jpg")
img = transforms(img)
img = img.unsqueeze(0) # produces a batch length of 1

prediction = simplenet(img)
prediction = prediction.argmax()
print(labels[prediction])

cat


## Saving and loading

In [58]:
torch.save(simplenet, "simplenet")

In [59]:
simplenet = torch.load("simplenet")

Can also use a state dict

In [60]:
torch.save(simplenet.state_dict(), "simplenet_dict")

In [61]:
simplenet = SimpleNet()
simplenet_state_dict = torch.load("simplenet_dict")
simplenet.load_state_dict(simplenet_state_dict)

<All keys matched successfully>